The ending of an URL usually contains a slug and and ID, usually, the slug will be ignored by website, the only thing matters is the ID

why not use it to exploit the entire webpage

In [ ]:
def download(url,user_agent='naruto',number_retries = 2, charset='utf-8'):
    print("Downloading:", url)
    request = urllib.request.Request(url)
    request.add_header("User-agent", user_agent)
    try:
        resp = urllib.request.urlopen(request)
        #verify if web charset is utf-8, if not ,we'll use utf-8 to decode, haha, 
        #it will throw error if no cs returned, but
        #we hope utf-8 will help
        #read more on pypi.python.org/pypi/chardet to implement a more robust decoder
        cs = resp.headers.get_content_charset()
        if not cs:
            cs = charset
        html = resp.read().decode(cs)
    except (URLError, HTTPError, ContentTooShortError) as e:
        print('Download error:', e.reason)
        html = None
        if number_retries > 0:
            if hasattr(e, "code") and 500 <= e.code <= 600:
                #recursive download process
                return download(url,number_retries -1)
    return html

In [1]:
import itertools

def crawl_site(url):
    for page in itertools.count(1):
        pg_url = '{}{}'.format(url,page)
        html = download(pg_url)
        if html is None:
            break
        # success - can scrape the result
        

one weakness of the above code is that some records may have been deleted, leaving gaps in the database IDs. Then when one of thesee gaps is reached, the crawler will immediately exit. 

let's see an improved version

In [5]:
def crawl_site(url, max_errors = 5):
    for page in itertools.count(1):
        pg_url = '{}{}'.format(url,page)
        html = download(pg_url)
        
        if html is None:
            num_errors += 1
            if num_errors == max_error:
                break
        else:
            num_errors =0
        

anyway, a lot of website don't use this way to download the data, for example amazon use ISBN as an ID, which is not pratical if we use the above method

next, I'm introducing the link crawlers

In [2]:
from bs4 import BeautifulSoup